In [1]:
import pandas as pd
import os
import numpy as np
import re
import datetime


In [2]:
FLOORING_CODE = ['FAA100', 'FAA150', 'FAA200', 'FCA100', 'FCA120', 'FCA160', 'FCA180',
       'FCA200', 'FCA300', 'FCA400', 'FCA500', 'FMS100', 'FMS110', 'FMS120',
       'FMS150', 'FMS200', 'FMS500', 'FMS600', 'FPO100', 'FPO200', 'FVN100',
       'FVN120', 'FVN150', 'FVN200', 'FVN210', 'FVN300', 'FVN350', 'FVN500',
       'FVN510', 'FVN550']

CLEANING_CODE = ['EHT520', 'KAA100', 'KAA200', 'KCH010', 'KCH020', 'KCH030', 'KCH050', 'KCH100',
       'KCH105', 'KCH110', 'KCH115', 'KCH140', 'KCH150', 'KCH160', 'KCH165',
       'KCH170', 'KCH180', 'KCH185', 'KCH200', 'KCH210', 'KCH220', 'KCH250',
       'KCH260', 'KCH270', 'KCH300', 'KCH330', 'KCH350', 'KCH360', 'KCH380',
       'KCH400', 'KCH505', 'KCH520', 'KCX100', 'KCX110', 'KCX150', 'KCX200',
       'KCX210', 'KCX220', 'KCX250', 'KCX300', 'KCX310', 'KCX320', 'TAA200',
       'TAA300', 'TAA310', 'TAA320', 'TAA330', 'TAA340', 'TAA350', 'TAA360',
       'TAA370', 'TAS100', 'TAS110', 'TAS120', 'TAS130', 'TCC100', 'TCC110',
       'TCC120', 'TCC130', 'TPC100', 'TPC115', 'TPC120', 'TPC130', 'TPC140',
       'TPC150', 'TRB100', 'TRB110', 'TRB120', 'TRB130', 'TRB140', 'TRB200',
       'TRC100', 'TRC110', 'TRC120', 'TRC130', 'TRR100', 'TRR110', 'TRR120',
       'TRS100', 'TRS110', 'TRS120', 'TRS130']

PAINTER_CODE = ['DAA100', 'DAA200', 'DAP100', 'DAP110', 'DAP120', 'DAP200', 'DAP210',
       'DPC100', 'DPC110', 'DPC120', 'DPC130', 'DPC140', 'DPC150', 'DPC152',
       'DPC160', 'DPC170', 'DPC180', 'DPC190', 'DPC200', 'DPC210', 'DPC250',
       'DPC300', 'DPI100', 'DPI150', 'DPI200', 'DPI300', 'DPI400', 'DPI410',
       'DPI420', 'DPI430', 'DPI440', 'DPI500', 'DPI510', 'DPI550', 'DPI800',
       'DPI900', 'DPI910', 'DPI920', 'DPL100', 'DPL110', 'DPL120', 'DPL130',
       'DPW100', 'DPW105', 'DPW110', 'DPW120', 'DPW130', 'DPW140', 'DPW150',
       'DPW152', 'DPW160', 'DPW170', 'DPW180', 'DPW190', 'DPW200', 'DPW210',
       'DRH100', 'DRH200', 'DRH300', 'DRH400', 'DRH500', 'DRP100', 'DRP105',
       'DRP110', 'DRP120', 'DRP130', 'DRP140', 'DRP150', 'DRP152', 'DRP160',
       'DRP170', 'DRP180', 'DRP190', 'DRP200', 'DRP250', 'DRA100', 'DRA200',
       'DPE110', 'DPE112', 'DPE115', 'DPE120', 'DPE130', 'DPE140', 'DPE150',
       'DPE160', 'DPE200', 'DPE201', 'DPE205', 'DPE210', 'DPE250', 'DPE400',
       'DPE410', 'DPE420', 'DPE460', 'DPE470', 'DPE480', 'DPE500', 'DPE510',
       'DPE550', 'DPE600', 'DPE650', 'DPE750', 'DPE760', 'DPE770', 'DPE780',
       'DPE800', 'DRE090', 'DRE100', 'DRE200', 'DRE300', 'DRE310', 'DRE400',
       'DRE410', 'DRE600', 'DRE700', 'DRE800', 'DRE900']

CARPENTER_CODE = ['BAA100', 'BAA200', 'BWK100', 'BWK200', 'BWK300', 'CAA100', 'CAA200',
       'CBT140', 'CBT200', 'CBT210', 'CDA100', 'CDA110', 'CDA200', 'CDA210',
       'CDA220', 'CDA300', 'CDA500', 'CDA600', 'CDT100', 'CDT110', 'CDT120',
       'CDT200', 'CDT210', 'CDT300', 'CDT400', 'CDT500', 'CDT510', 'CDT520',
       'CDT600', 'CDT610', 'CDT650', 'CDT660', 'CDT700', 'CDT710', 'CDT750',
       'CDT760', 'CDT770', 'CDT800', 'CDT810', 'CDT820', 'CDT900', 'CDX100',
       'CDX150', 'CDX160', 'CDX190', 'CDX200', 'CDX210', 'CDX250', 'CDX260',
       'CDX300', 'CDX310', 'CFE300', 'CFR080', 'CFR100', 'CFR110', 'CFR120',
       'CFR130', 'CFR140', 'CFR150', 'CFR200', 'CFR210', 'CFR300', 'CFR340',
       'CFR550', 'CFR560', 'CFR600', 'CHB100', 'CHB200', 'CHB220', 'CHB250',
       'CHB300', 'CHB310', 'CHB320', 'CHB330', 'CHB350', 'CHB400', 'CHB450',
       'CHC100', 'CHC200', 'CHC220', 'CHC230', 'CHC300', 'CHC320', 'CHC400',
       'CHC450', 'CHD100', 'CHD105', 'CHD110', 'CHD120', 'CHD125', 'CHD130',
       'CHD140', 'CHD150', 'CHD160', 'CHD200', 'CHD205', 'CHD215', 'CHD300',
       'CHD305', 'CHD310', 'CHD320', 'CHD321', 'CHD322', 'CHD330', 'CHD340',
       'CHD345', 'CHD350', 'CHD360', 'CHD380', 'CHD385', 'CHD400', 'CHD410',
       'CHD415', 'CHD435', 'CHD460', 'CHD470', 'CHD480', 'CHD490', 'CHD500',
       'CHD510', 'CHD520', 'CHD530', 'CHD540', 'CHD700', 'CHD750', 'CHD760',
       'CHD770', 'CHD800', 'CHD810', 'CHD900', 'CHD910', 'CHM100', 'CHM150',
       'CHM200', 'CHM210', 'CHM264', 'CHM268', 'CHM270', 'CHM283', 'CHM285',
       'CHM290', 'CHM292', 'CHM294', 'CHM295', 'CHM296', 'CHM300', 'CHM400',
       'CHM500', 'CHW100', 'CHW110', 'CHW120', 'CHW150', 'CHW160', 'CHW170',
       'CHW200', 'CHW205', 'CHW300', 'CHW310', 'CHW400', 'CHW410', 'CHW420',
       'CHW430', 'CHW450', 'CHX100', 'CHX150', 'CIN110', 'CIN150', 'CIN151',
       'CIN152', 'CIN180', 'CIN210', 'CIN230', 'CIN235', 'CIN250', 'CIN300',
       'CIN305', 'CIN400', 'CIN610', 'CIN620', 'CJC050', 'CJC100', 'CJC110',
       'CJC240', 'CJC250', 'CJC260', 'CJC270', 'CJC350', 'CJC355', 'CJC360',
       'CJC370', 'CJC380', 'CJC390', 'CJC500', 'CJC800', 'CJD100', 'CJD110',
       'CJD150', 'CJD160', 'CJD200', 'CJD210', 'CJD250', 'CJD260', 'CJD450',
       'CJD500', 'CJD600', 'CJD610', 'CJD630', 'CJD650', 'CJD660', 'CJD700',
       'CJD800', 'CLI010', 'CLI020', 'CLI055', 'CLI060', 'CLI080', 'CLI090',
       'CLI100', 'CLI110', 'CLI150', 'CLI160', 'CLI180', 'CLI190', 'CLI200',
       'CLI300', 'CLI400', 'CLI450', 'CLI460', 'CLI500', 'CLI550', 'CLW100',
       'CLW200', 'CLW210', 'CLW300', 'CLW400', 'CLW410', 'CLW500', 'CLX100',
       'CLX110', 'CLX150', 'CLX200', 'CLX210', 'CLX250', 'CLX300', 'CLX310',
       'CLX350', 'CLX360', 'CLX400', 'CLX600', 'CLX700', 'CLX750', 'CLX760',
       'CLX800', 'CMS010', 'CMS020', 'CMS050', 'CMS060', 'CMS070', 'CMS100',
       'CMS110', 'CMS120', 'CMS150', 'CMS170', 'CMS180', 'CMS190', 'CMS200',
       'CMS220', 'CMS400', 'CMS420', 'CMS500', 'CMS600', 'CMS610', 'CMS700',
       'CMS705', 'CMS715', 'CMS717', 'CMS720', 'CMS722', 'CMS725', 'CMS760',
       'CMS770', 'CMS800', 'CMS900', 'CSA100', 'CSA200', 'CSA500', 'CSA600',
       'CSA700', 'CSB100', 'CSB200', 'CSB250', 'CSB300', 'CSB400', 'CSB500',
       'CSB505', 'CSB510', 'CSB600', 'CSB650', 'CSH100', 'CSH200', 'CSH310',
       'CSR100', 'CSR120', 'CSR150', 'CSR170', 'CSR200', 'CTF100', 'CTF200',
       'CTF210', 'CTF250', 'CTF260', 'CTF300', 'CTF310', 'CTF320', 'CTI100',
       'CTI160', 'CTI200', 'CTI210', 'CTI220', 'CTI300', 'CTI310', 'CTI320',
       'CTI350', 'CTI400', 'CTI410', 'CTX100', 'CTX300', 'CTX310', 'CTX320',
       'CTX330', 'CTX400', 'CTX410', 'CTX420', 'CTX430', 'CVX100', 'CVX200',
       'CVX210', 'CVX250', 'CWA100', 'CWA110', 'CWA150', 'CWA220', 'CWA230',
       'CWA250', 'CWA320', 'CWA350', 'CWA600', 'CWA800', 'CWT050', 'CWT060',
       'CWT100', 'CWT200', 'CWT210', 'CWT300', 'CWT310', 'CWT400', 'CWT450',
       'CWT460', 'CWT470', 'CWT600', 'YAA100', 'YAA200', 'YAA300', 'YAA400',
       'YAA500', 'YAA600', 'YCL100', 'YCL150', 'YCL200', 'YCL210', 'YCL220',
       'YCL230', 'YCL250', 'YCL300', 'YCL310', 'YCL400', 'YCL405', 'YCL450',
       'YCL455', 'YCL500', 'YCX500', 'YCX550', 'YCX560', 'YCX570', 'YCX700',
       'YCX900', 'YCX910', 'YCX950', 'YFE050', 'YFE055', 'YFE060', 'YFE070',
       'YFE100', 'YFE110', 'YFE200', 'YFE210', 'YFE220', 'YFE230', 'YFE240',
       'YFE250', 'YFE260', 'YFE300', 'YFE310', 'YFE320', 'YFE355', 'YFE365',
       'YFE370', 'YFE380', 'YFE400', 'YFE410', 'YFE420', 'YFE460', 'YFE480',
       'YFE500', 'YFE510', 'YFE520', 'YFE580', 'YFE610', 'YFE630', 'YFE650',
       'YFE800', 'YFE830', 'YFE850', 'YGT050', 'YGT100', 'YGT110', 'YGT120',
       'YGT130', 'YGT150', 'YGT180', 'YGT200', 'YGT210', 'YGT220', 'YGT300',
       'YGT310', 'YGT320', 'YGT400', 'YGT410', 'YGT420', 'YGT430', 'YGT800',
       'YGT810', 'YGT820', 'YLB100', 'YLB110', 'YLB130', 'YLB150', 'YLB230',
       'YLB240', 'YLB245', 'YLB250', 'YLB251', 'YLB310', 'YLB320', 'YLB330',
       'YLB400', 'YMG205', 'YMG230', 'YMG235', 'YMG240', 'YMG245', 'YMG250',
       'YMG260', 'YMG270', 'YMG280', 'YMG285', 'YMG300', 'YMG400', 'YMG540',
       'YMG550', 'YMG555', 'YMG560', 'YMG565', 'YMG570', 'YMG580', 'YMS100',
       'YMS200', 'YMS210', 'YMS220', 'YMS300', 'YPV050', 'YPV060', 'YPV080',
       'YPV091', 'YPV100', 'YPV110', 'YPV150', 'YPV160', 'YPV200', 'YPV210',
       'YPV250', 'YPV260', 'YPV300', 'YPV380', 'YPV390', 'YPV400', 'YPV410',
       'YPV420', 'YPV500', 'YPV600', 'GTW610','YPV610', 'CJC355', ]

DATE = datetime.datetime.now()
INVOICE_DATE = "%s/%s/%s"%(DATE.day, DATE.month, DATE.year)
DUE_DATE = "%s/%s/%s"%(DATE.day, DATE.month +1, DATE.year)
FILE_NAME_DATE = "%s-%s-%s"%(DATE.day, DATE.month, DATE.year)

FILE = "R5497282-3 40 GEAR TCE"

In [3]:
target_df = pd.read_csv("%s.csv"%FILE)


whole_df = pd.DataFrame(columns=target_df.columns)
flooring_df = pd.DataFrame(columns=target_df.columns)
carpenter_df =  pd.DataFrame(columns=target_df.columns)
painter_df =  pd.DataFrame(columns=target_df.columns)
cleaning_df =  pd.DataFrame(columns=target_df.columns)

In [4]:
working_df = pd.read_csv("%s.csv"%FILE)
print(working_df.columns)
working_df = working_df[['WorkOrder', 'StreetAddress3', 'ItemCode', 'quantity1',]]
working_df

Index(['WorkOrder', 'PICCNO', 'Textbox8', 'Textbox9', 'Textbox10',
       'Folio_Name', 'StreetAddress', 'StreetAddress3', 'StreetAddress2',
       'Textbox19', 'StreetAddress5', 'Textbox18', 'NotesToTP',
       'StreetAddress4', 'Textbox16', 'StreetAddress6', 'Textbox68',
       'Textbox22', 'Textbox24', 'Textbox26', 'Work_Onsite_Date_Time',
       'Work_Completed_Date_Time', 'Textbox50', 'TradeDescription', 'ItemCode',
       'quantity1', 'Textbox15', 'Textbox6', 'StreetAddress11',
       'StreetAddress13', 'StreetAddress15', 'Work_Done_Text', 'Textbox76',
       'Textbox77'],
      dtype='object')


,WorkOrder,StreetAddress3,ItemCode,quantity1
0,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,CBA120,1.00
1,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,CBA200,1.00
2,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,CFR300,4.75
3,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,CHB320,1.00
4,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,CHD500,1.00
...,...,...,...,...
108,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,DRH300,1.00
109,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,KCH050,0.50
110,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,KCH180,1.00
111,R5497282-3,40 GEAR TCE PORIRUA EAST PORIRUA,KCX210,1.00


In [5]:
quote_template_df = pd.read_excel("Downer Quote Template 2023-2024 Responsive.xlsx", sheet_name="ItemList")
def lookup_table(item_code):
    for idx,row in quote_template_df.iterrows():
        if row['Rentel'] == item_code:
            return row

quote_template_df        

,Rentel,Short Description,Long Description,Unit of Measure,Value incl GST at 15%
0,AAD400,GENERAL CONSULTANCY,General consultancy.,$,1.0005
1,ABS050,BWOF COSTS,Costs for managing the Building Warrant of Fit...,$,1.0005
2,ABS100,"AUTO DOORS , GENERAL",Automatic door maintenance - general.,$,1.0005
3,ABS300,"LIFTS, GENERAL",General lift maintenance.,$,1.0005
4,ACR170,"SECURITY SYSTEM, GENERAL REPR",General repairs to security system.,$,1.0005
...,...,...,...,...,...
1446,YPV610,"PAVE 150 FTPATH CROSSING, LAY",Supply and lay 150 thick driveway footpath cro...,M2,249.7300
1447,YXM100,PLAYGROUND MAINTENANCE,Maintenance and repairs of playgrounds.,$,1.0005
1448,YXM200,GROUNDS MAINTENANCE,Communal ground maintenance as per scope of wo...,$,1.0005
1449,YXM210,PLANT REPLACE,Remove and replace plants,$,1.0005


In [6]:
def update_df(target_df, reference, inventory_item_code, quantity, description, unit_amount):
    target_df = target_df.append({"Reference": reference,"InventoryItemCode": "%s new"%(inventory_item_code), "*Quantity": quantity,
                                 "*InvoiceDate": INVOICE_DATE, "*DueDate":DUE_DATE, "*ContactName": "Spotless",
                                 "*AccountCode": "200", "*TaxType": "No Tax", "*Description":description, 
                                 "*UnitAmount": unit_amount , "Currency":"NZD"}
                                 ,ignore_index=True )
    return target_df

In [7]:
for idx,row in working_df.iterrows():
    reference = "%s - %s"%(row["WorkOrder"],row["StreetAddress3"])
    inventory_item_code = row['ItemCode'].strip()
    quantity = row['quantity1']
    
    if re.search("^IC\d+$", inventory_item_code):
        print(inventory_item_code)
        continue
    
    print(inventory_item_code)
    quote_template_row = lookup_table(inventory_item_code)
    if quote_template_row.empty:
        continue
    
    description = quote_template_row["Short Description"]
    unit_amount = quote_template_row["Value incl GST at 15%"]
    
    if inventory_item_code in FLOORING_CODE:
        
        flooring_df = update_df(flooring_df,  reference, inventory_item_code, quantity, description, unit_amount)
        whole_df = update_df(whole_df,  reference, inventory_item_code, quantity, description, unit_amount)
        
    elif inventory_item_code in CLEANING_CODE:
        cleaning_df = cleaning_df.append({"Reference": reference,"InventoryItemCode": "%s new"%(inventory_item_code), "*Quantity": quantity,
                                 "*InvoiceDate": INVOICE_DATE, "*DueDate":DUE_DATE, "*ContactName": "Spotless",
                                 "*AccountCode": "200", "*TaxType": "No Tax", "*Description":description, 
                                 "*UnitAmount": unit_amount , "Currency":"NZD"}
                                 ,ignore_index=True )
        whole_df = update_df(whole_df,  reference, inventory_item_code, quantity, description, unit_amount)
        
    elif inventory_item_code in CARPENTER_CODE:
        carpenter_df = carpenter_df.append({"Reference": reference,"InventoryItemCode": "%s new"%(inventory_item_code), "*Quantity": quantity,
                                 "*InvoiceDate": INVOICE_DATE, "*DueDate":DUE_DATE, "*ContactName": "Spotless",
                                 "*AccountCode": "200", "*TaxType": "No Tax", "*Description":description, 
                                 "*UnitAmount": unit_amount , "Currency":"NZD"}
                                 ,ignore_index=True )
        whole_df = update_df(whole_df,  reference, inventory_item_code, quantity, description, unit_amount)

    elif inventory_item_code in PAINTER_CODE:
        painter_df = painter_df.append({"Reference": reference,"InventoryItemCode": "%s new"%(inventory_item_code), "*Quantity": quantity,
                                 "*InvoiceDate": INVOICE_DATE, "*DueDate":DUE_DATE, "*ContactName": "Spotless",
                                 "*AccountCode": "200", "*TaxType": "No Tax", "*Description":description, 
                                 "*UnitAmount": unit_amount , "Currency":"NZD"}
                                 ,ignore_index=True )
        whole_df = update_df(whole_df,  reference, inventory_item_code, quantity, description, unit_amount)


CBA120
CBA200
CFR300
CHB320
CHD500
CLW200
CLW210
DAP100
FMS150
FVN100
FVN200
FVN500
FVN550
CDT500
CDT650
CHD105
CHD140
CHD500
CHM300
CLI200
DAA100
DAP100
DPI300
FCA300
FCA400
CDT500
CDT650
CHD105
CHD140
CHD500
CHM300
CHW100
CHW150
CHW400
CLI200
DAP100
DPI300
DPL110
DPL120
FCA300
FCA400
CDT700
CHD105
CHD140
CHD200
CHM300
CHW100
CHW400
DAP100
DPL120
FCA300
FCA400
KCX110
KCH330
CHD100
CHM300
CLI200
DAP100
DPL120
FCA300
FCA400
CHD800
FCA300
FCA400
CAA100
CHC400
CHD500
CJD200
CLI200
DAP100
DPI420
DPI440
KCH260
KCH270
KCH330
CHD500
CLW100
DAP100
DPE140
KCH330
CHD500
CLI200
DAP100
DPL110
DPL120
FCA300
FCA400
CSB500
KCX200
YFE380
YFE610
YGT180
YLB110
YLB150
CLI200
DAP100
DPI920
DPL120
FCA400
FCA500
CHD500
CHD540
DAP100
FVN100
FVN200
FVN500
FVN550
AES100
DRH300
KCH050
KCH180
KCX210
KCX250


In [8]:
flooring_df

,WorkOrder,PICCNO,Textbox8,Textbox9,Textbox10,Folio_Name,StreetAddress,StreetAddress3,StreetAddress2,Textbox19,...,*ContactName,*Description,*DueDate,*InvoiceDate,*Quantity,*TaxType,*UnitAmount,Currency,InventoryItemCode,Reference
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"FL COVERING UNDERLAY, S&F",6/12/2023,6/11/2023,3.10,No Tax,51.12700,NZD,FMS150 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"VINYL, REM",6/12/2023,6/11/2023,3.10,No Tax,13.35150,NZD,FVN100 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"VINYL <=4M2, LAY",6/12/2023,6/11/2023,3.10,No Tax,82.49550,NZD,FVN200 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"VINYL 100MM HIGH SKIRTING, LAY",6/12/2023,6/11/2023,6.00,No Tax,24.94000,NZD,FVN500 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"VINYL, AL COVE CAPPING",6/12/2023,6/11/2023,6.00,No Tax,26.00425,NZD,FVN550 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"CARPET, LAY",6/12/2023,6/11/2023,14.75,No Tax,45.63375,NZD,FCA300 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"CARPET UNDERLAY, LAY",6/12/2023,6/11/2023,14.75,No Tax,10.82525,NZD,FCA400 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"CARPET, LAY",6/12/2023,6/11/2023,9.75,No Tax,45.63375,NZD,FCA300 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"CARPET UNDERLAY, LAY",6/12/2023,6/11/2023,9.75,No Tax,10.82525,NZD,FCA400 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Spotless,"CARPET, LAY",6/12/2023,6/11/2023,8.25,No Tax,45.63375,NZD,FCA300 new,R5497282-3 - 40 GEAR TCE PORIRUA EAST PORIRUA


In [9]:
flooring_df = flooring_df.to_csv("%s_Flooring_SalesInvoice_%s.csv"%(FILE, FILE_NAME_DATE), index=False)
carpenter_df = carpenter_df.to_csv("%s_Carpenter_SalesInvoice_%s.csv"%(FILE, FILE_NAME_DATE), index=False)
painter_df = painter_df.to_csv("%s_Painter_SalesInvoice_%s.csv"%(FILE, FILE_NAME_DATE), index=False)
cleaning_df =cleaning_df.to_csv("%s_Cleaning_SalesInvoice_%s.csv"%(FILE, FILE_NAME_DATE), index=False)

whole_df = whole_df.to_csv("%s_whole_SalesInvoice_%s.csv"%(FILE, FILE_NAME_DATE), index=False)

In [10]:
CURRENT_DIR = os.getcwd()
FILES = []

for file in os.listdir(CURRENT_DIR):
    if re.search('^ServiceOrder(.*?).csv',file):
        FILES.append(file)


In [11]:
def load_preprocessing(input_dir):
    raw_data_df = pd.read_excel(input_dir)
    select_columns = ['Shear rate','Torque', 'Stress']
    preprocessed_data_df = raw_data_df.copy()
    preprocessed_data_df = preprocessed_data_df.drop(index=0, axis=0)
    return preprocessed_data_df

In [12]:
def process_data(preprocessed_data_df):
    output_df_dict = {'Shear rate 1/s':[], 'Torque µN.m':[], 'Torque N.m':[], 'Stress MPa':[], 'Stress Pa':[]}
    
    for i in range(preprocessed_data_df.shape[0]):
        row = preprocessed_data_df.iloc[i]
        share_rate, torque_unm, stress_mpa = row['Shear rate'],row['Torque'], row['Stress']
        torque_nm, stress_pa =  torque_unm * (10**-6), stress_mpa * (10**6)
        output_df_dict['Shear rate 1/s'].append(np.around(share_rate,decimals=1))
        output_df_dict['Torque µN.m'].append(torque_unm)
        output_df_dict['Torque N.m'].append(torque_nm)
        output_df_dict['Stress MPa'].append(stress_mpa)
        output_df_dict['Stress Pa'].append(stress_pa)
    
    output_df = pd.DataFrame(output_df_dict)
    
    return output_df

In [13]:
# for file in FILES:
#     file_dir = os.path.join(CURRENT_DIR, file)
#     output_dir = os.path.join(CURRENT_DIR, 'parsed ' + file)

#     preprocessed_data_df = load_preprocessing(file_dir)
#     output_df = process_data(preprocessed_data_df)
#     output_df.to_excel(output_dir,index=False)
